<a href="https://colab.research.google.com/github/michaelhlew/nlp-project/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle

In [ ]:
mkdir ~/.kaggle

In [ ]:
cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d leadbest/googlenewsvectorsnegative300

100% 3.16G/3.17G [00:33<00:00, 155MB/s]
100% 3.17G/3.17G [00:33<00:00, 102MB/s]


In [ ]:
!unzip googlenewsvectorsnegative300.zip

Archive:  googlenewsvectorsnegative300.zip
  inflating: GoogleNews-vectors-negative300.bin  
  inflating: GoogleNews-vectors-negative300.bin.gz  


In [9]:
# import libraries
import pandas as pd
import numpy as np
import re
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet, stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import NMF, PCA, TruncatedSVD, FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, KFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
from bs4 import BeautifulSoup as bs
from nltk.util import ngrams
import cgi
import nltk
import html
#import readability 
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Data Pre-Processing**

In [4]:
biden = pd.read_csv("biden_filtered.csv")
trump = pd.read_csv("trump_filtered.csv")
biden['author'] = "Biden"
trump['author'] = "Trump"

In [5]:
trump = trump[trump.columns[1:]]
biden = biden[biden.columns[1:]]

In [6]:
tweets = pd.concat([biden, trump])

In [7]:
class TwitterCleaner:
    def __init__(self, df, column):
        self.df = df
        self.column = column

    def decode_xml_entities(self, text):
        text = html.unescape(text)
        text = str(text)
        return text

    def clean_text(self, lemmatize=False, stem=False, remove_stopwords=True):
        stop_words = set(stopwords.words('english'))
        self.df[self.column] = self.df[self.column].apply(lambda x: str(x))
        clean_list = []
        for text in self.df[self.column]:
            # remove URLs
            text = re.sub(r'http\S+','', text)  

            # remove '@' twitter mentions
            text = re.sub(r'@[A-Za-z0-9_]+','', text) 

            # XML to characters
            text = self.decode_xml_entities(text)

            # lowercase the text
            text = text.lower() 
            words = word_tokenize(text)

            clean_words = []

            # stemming / lemmatization

            for word in words:
                if remove_stopwords == True and word in stop_words:
                    continue
                if lemmatize == True: 
                    lemmatizer = WordNetLemmatizer()
                    word = lemmatizer.lemmatize(word)
                if stem == True: 
                    stemmer = PorterStemmer()
                    word = stemmer.stem(word)
                clean_words.append(word)

            clean_text = ' '.join(clean_words)
            clean_list.append(clean_text)

        if lemmatize:
            if remove_stopwords:
                self.df[f"{self.column}_clean_lemmatized_stopwords"] = clean_list
            else:
                self.df[f"{self.column}_clean_lemmatized"] = clean_list
        elif stem:
            if remove_stopwords:
                self.df[f"{self.column}_clean_stemmed_stopwords"] = clean_list
            else:
                self.df[f"{self.column}_clean_stemmed"] = clean_list
        else:
            if remove_stopwords:
                self.df[f"{self.column}_clean_stopwords"] = clean_list
            else:
                self.df[f"{self.column}_clean"] = clean_list

        return self.df


In [13]:
tc = TwitterCleaner(tweets, 'text')
tweets = tc.clean_text(lemmatize = True, stem = False, remove_stopwords = True)
tweets = tc.clean_text(lemmatize = False, stem = True, remove_stopwords = True)
tweets

,text,likes,retweets,timestamp,id,author,text_clean_lemmatized_stopwords,text_clean_stemmed_stopwords
0,Every single human being deserves to be treate...,11574,2423,2020-01-01 18:35:00,1.212442e+18,Biden,every single human deserves treated dignity . ...,everi singl human deserv treat digniti . every...
1,With just over one month until the Iowa Caucus...,1457,368,2020-01-02 00:01:00,1.212524e+18,Biden,"one month iowa caucus , need hand deck talk fo...","one month iowa caucu , need hand deck talk fol..."
2,This election is about the soul of our nation ...,44886,10192,2020-01-02 01:05:00,1.212540e+18,Biden,election soul nation — donald trump poison soul .,elect soul nation — donald trump poison soul .
3,Every day that Donald Trump remains in the Whi...,9581,2005,2020-01-02 02:07:00,1.212556e+18,Biden,every day donald trump remains white house put...,everi day donald trump remain white hous put f...
4,It was a privilege to work with @JulianCastro ...,17156,2284,2020-01-02 16:10:00,1.212768e+18,Biden,"privilege work obama administration , true hon...","privileg work obama administr , true honor tal..."
...,...,...,...,...,...,...,...,...
6403,"Iran never won a war, but never lost a negotia...",303007,57253,2020-01-03 12:44:30,1.213079e+18,Trump,"iran never war , never lost negotiation !","iran never war , never lost negoti !"
6404,Thank you to the @dcexaminer Washington Examin...,35044,9213,2020-01-01 01:03:15,1.212177e+18,Trump,thank washington examiner . list growing every...,thank washington examin . list grow everi day !
6405,One of my greatest honors was to have gotten C...,56731,12761,2020-01-01 00:55:01,1.212175e+18,Trump,one greatest honor gotten choice approved grea...,one greatest honor gotten choic approv great v...
6406,Just signed an order to support the workers of...,176289,36001,2020-10-22 21:04:21,1.319384e+18,Trump,signed order support worker delphi corporation...,sign order support worker delphi corpor make s...


In [12]:
tweets.groupby(['author'])['likes'].mean()

author
Biden     63039.107277
Trump    117850.719101
Name: likes, dtype: float64

In [15]:
tweets.groupby(['author'])['retweets'].mean()

author
Biden    11243.899898
Trump    25571.497035
Name: retweets, dtype: float64

**Analysis**

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

def sentiment_score(sentence):
    score = analyzer.polarity_scores(sentence)
    return score['compound']

tweets['sentiment_scores'] = [sentiment_score(sentence) for sentence in tweets['text_clean_lemmatized_stopwords']]
tweets

In [26]:
from textblob import TextBlob
# function to calculate subjectivity
def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity
    # function to calculate polarity
def getPolarity(review):
    return TextBlob(review).sentiment.polarity
# function to analyze the reviews
def analysis(score):
    if score < 0:
        return '0' #Negative 
    else:
        return '1' #Positive

In [28]:
tweets['Subjectivity'] = tweets['text_clean_lemmatized_stopwords'].apply(getSubjectivity) 
tweets['Polarity'] = tweets['text_clean_lemmatized_stopwords'].apply(getPolarity) 
tweets['Analysis'] = tweets['Polarity'].apply(analysis)
tweets.head()

,text,likes,retweets,timestamp,id,author,text_clean_lemmatized_stopwords,text_clean_stemmed_stopwords,Subjectivity,Polarity,Analysis
0,Every single human being deserves to be treate...,11574,2423,2020-01-01 18:35:00,1.212442e+18,Biden,every single human deserves treated dignity . ...,everi singl human deserv treat digniti . every...,0.452381,-0.295238,0
1,With just over one month until the Iowa Caucus...,1457,368,2020-01-02 00:01:00,1.212524e+18,Biden,"one month iowa caucus , need hand deck talk fo...","one month iowa caucu , need hand deck talk fol...",0.000000,0.000000,1
2,This election is about the soul of our nation ...,44886,10192,2020-01-02 01:05:00,1.212540e+18,Biden,election soul nation — donald trump poison soul .,elect soul nation — donald trump poison soul .,0.000000,0.000000,1
3,Every day that Donald Trump remains in the Whi...,9581,2005,2020-01-02 02:07:00,1.212556e+18,Biden,every day donald trump remains white house put...,everi day donald trump remain white hous put f...,0.075000,0.033333,1
4,It was a privilege to work with @JulianCastro ...,17156,2284,2020-01-02 16:10:00,1.212768e+18,Biden,"privilege work obama administration , true hon...","privileg work obama administr , true honor tal...",0.516667,0.350000,1


In [ ]:
print("Average sentiment score for Biden: " + str(tweets[tweets['author'] == "Biden"]['sentiment_scores'].mean()))

Average sentiment score for Biden: 0.17794974376494702


In [ ]:
print("Average sentiment score for Trump: " + str(tweets[tweets['author'] == "Trump"]['sentiment_scores'].mean()))

Average sentiment score for Trump: 0.1705028870162297


In [ ]:
# Basically the same sentiment score 

In [ ]:
#CNN
import random
random.seed(123)
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D
#NN imports
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model, Sequential
from keras.initializers import Constant

from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from sklearn.preprocessing import LabelEncoder

In [ ]:
#Recoding author data into binary
tweets['author_dummy'] = np.where(tweets['author'] == "Biden", 1, 0)
tweets.head()

In [ ]:
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit=200000)
print('done loading Word2Vec')

done loading Word2Vec


In [ ]:
def tokenize_and_vectorize(tweets):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []
    for sample in tweets:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])

            except KeyError:
                pass  # No matching token in the Google w2v vocab
            
        vectorized_data.append(sample_vecs)

    return vectorized_data

In [ ]:
def collect_expected(tweets):
    """ Peel of the target values from the dataset """
    expected = ['author_dummy']
    for sample in tweets:
        expected.append(sample[0])
    return expected

In [ ]:
vectorized_data = tokenize_and_vectorize(tweets)
expected = collect_expected(tweets)

In [ ]:
# Split data
split_point = int(len(vectorized_data)*.7)

x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

In [ ]:
maxlen = 400
embedding_dims = 300  

In [ ]:
 # Must manually pad/truncate

def pad_trunc(data, maxlen):
    """ For a given dataset pad with zero vectors or truncate to maxlen """
    new_data = []

    # Create a vector of 0's the length of our word vectors
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)

    for sample in data:
 
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [ ]:
x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))

y_train = np.array(y_train)

x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))

y_test = np.array(y_test)

In [ ]:
batch_size = 32         # How many samples to show the net before backpropogating the error and updating the weights
filters = 250           # Number of filters we will train
kernel_size = 3         # The width of the filters, actual filters will each be a matrix of weights of size: embedding_dims x kernel_size or 50 x 3 in our case
hidden_dims = 250       # \
epochs = 3  

In [ ]:
print('Build model...')
model = Sequential()

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1,
                 input_shape=(maxlen, embedding_dims)))
# we use max pooling:
model.add(GlobalMaxPooling1D())
# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

Build model...


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Epoch 1/3


UnimplementedError: ignored